In [12]:
import cPickle as pickle
import numpy as np
from pprint import pprint

with open('thesis/results/features_2016-12-14_10-15-43.pkl') as f:
    data, configuration = pickle.load(f)

print "PROJECTS:"
for p_name in data.iterkeys():
    print p_name

print
print "CONFIG:"
pprint(configuration)

PROJECTS:
Sowbug3
Cam1
Camera3
Zebrafish

CONFIG:
{'rf_class_weight': 'balanced_subsample',
 'rf_criterion': 'entropy',
 'seed': 42,
 'test_split_method': 'random'}


In [22]:
p_names = filter(lambda x: x != 'layer', [p for p in data.iterkeys()])
print p_names
splits = filter(lambda x: x != 'layer', [s for s in data[p_names[0]].iterkeys()])
print splits
features = filter(lambda x: x!= 'layer', [f for f in data[p_names[0]][splits[0]].iterkeys()])
print features

# for p_name, data in data.iteritems():
#     print p_name

['Sowbug3', 'Cam1', 'Camera3', 'Zebrafish']
[0.95, 0.9, 0.8, 0.99]
['fm_lbp ', 'fm_colornames ', 'fm_idtracker_c_d50 ', 'fm_idtracker_i_d50 ', 'fm_basic ', 'fm_hog ']


In [ ]:
from tabulate import tabulate

for split in splits:
    print "------- ", split, " -------"
    
    table = []
    i = 0
    table.append([''])
    keys = []
    for feat in features:
        # remove fm_ p
        f = '_'.join(feat.split('_')[1:])
        table[i].append(f)
        keys.append(f)
        
    for p_name in p_names:
        i += 1
        print "###", p_name, "###"
        table.append([p_name])
        keys.append(p_name)
        for feat in features:
            it = data[p_name][split][feat]
                
            if isinstance(it, dict):
                s = "{:.2%} ({:.4f})".format(np.mean(it['accuracy']), np.std(it['accuracy']))
                table[i].append(s)
                # print feat, "\t", s
    
    print
    print tabulate(table, tablefmt='plain')
    print

In [20]:
print data['Sowbug3'].keys()

[0.95, 'layer', 0.9, 0.8, 0.99]
